### Monitoreo del área inundada de presas de México con Landsat

## Descripción

Esta es una notebook para el monitoreo del área inundada por el embalse de las principales presas de México utilizando imágenes de Landsat desde Google Earth Engine.

*Material didáctico que para obtener el título de Ingeniera Geofísica*

Google ha desarrollado Google Earth Engine, una plataforma que cuenta con un extenso catálogo de información derivada de diferentes misiones satelitales. La API para el lenguaje de programación Python permite acceder a la información con la que se realiza el cálculo de un estimado de la cantidad de agua almecenada en cada presa por un periodo de tiempo, los datos obtenidos del monitoreo satelital se capturan en series de tiempo que muestran el comportamiento de las principales presas de México.

## Acerca de los autores

- **Maria Isabel Celso Crescencio**: Ing. Geofísica por la Facultad de Ingeniería de la UNAM

- **Saúl Arciniega Esparza**: Profesor Asociado C de TC, Facultad de Ingeniería de la UNAM

In [1]:
# @title Importar librerias
# Importación de módulos
import json
import datetime
from time import sleep
import numpy as np
import pandas as pd
import ee
import geemap.foliumap as geemap
import eemont
from statistics import quantiles

import panel as pn
from io import StringIO
import plotly.offline as po
import plotly.express as pe
import plotly.graph_objects as go

pn.extension('plotly')
pn.extension(design='material')

import warnings
warnings.filterwarnings('ignore')

# Acceder a _Earth Engine_

### Instrucciones:

1. Ejecutar la celda con el título _"Inicializar Google Earth Engine API"_.
2. Acceder al link.
3. Seleccionar la cuenta _Google_ desde la que accesará. $\rightarrow$ Continuar.
4. Seleccionar la casilla _use **read-only scopes**_, posteriormente seleccionar _**GENERAR TOKEN**_.
5. Seleccionar cuenta _Google_ para acceder a _Earth Engine Notebook Client_.
6. Aparecerá un mensaje que indica que la aplicación no esta verificada por _Google_, seleccionar **Continuar**.
7. Seleccionar las casillas para permitir el acceso a _Earth Engine Notebook Client_. $\rightarrow$ Continuar.
8. Copiar el código de autorización proporcionado.
9. Pegar el código en la casilla que se ubica debajo del link en la notebook.

![Gee_guia](https://upload.wikimedia.org/wikipedia/commons/2/2f/Gee_guia.png)

In [2]:
# @title Autenticación para inicializar Google Earth Engine API
# Autenticación y credenciales para inicializar 
ee.Authenticate(force = True)

Enter verification code:  4/1AQSTgQHR8mk4CAgr7tTAZWsK3kuOQEaUdKRDTS651QV9qns5J0pysFunAII



Successfully saved authorization token.


In [3]:
# @title Inicialización
# Verificación de que existen credenciales válidas
ee.Initialize()

In [4]:
# @title Generar información de las presas

# Construcción de mapa con datos geograficos y de identidad
layer_file = r"./data/Presas.shp"
info_file = r"./data/presas_info.csv"
capa = geemap.shp_to_ee(layer_file)
info = pd.read_csv(info_file)

# Extracción de atributos de la capa
capa_atributos = capa.getInfo()
columna = "Clave"
campos = [item["properties"][columna] for item in capa_atributos["features"]]
areas = pd.Series(
    [item["properties"]["Area"] for item in capa_atributos["features"]],
    index = campos,
)

# Diccionario de claves
clave = {f"{info.iloc[i, 0]} - {info.iloc[i, 3]}": info.iloc[i, 1] for i in range(len(info))}

In [5]:
# @title Funciones de procesamiento

# Conversión de las series temporales por región 
# a series de tiempo en la librería pandas 
def gee_ts_to_serie(ts, var):
    info = ts.getInfo()
    data = []
    for feat in info["features"]:
        d = feat["properties"]
        data.append([d["date"], d[var]])
    df = pd.DataFrame(data, columns = ["date", var])
    df.set_index("date", inplace = True)
    df.index = pd.to_datetime(df.index)
    df[df == -9999] = np.nan
    df = df.dropna()
    dates = pd.to_datetime([f"{x.year}-{x.month}-{x.day}" for x in df.index])
    df.index = dates
    df.sort_index(inplace=True)
    return df

# Obtención de las series por región para la colección de imágenes y geometría dadas     
def collection_timeserie(collection, geom, band, scale=30, reducer="mean"):
    if reducer == "mean":
        reducer = ee.Reducer.mean()
    elif reducer == "sum":
        reducer = ee.Reducer.sum()
    elif reducer == "count":
        reducer = ee.Reducer.count()
    elif reducer == "covariance":
        reducer = ee.Reducer.covariance()
    elif reducer == "std":
        reducer = ee.Reducer.stdDev()
    elif reducer == "min":
        reducer = ee.Reducer.min(1)
    elif reducer == "max":
        reducer = ee.Reducer.max(1)

    ts = collection.getTimeSeriesByRegion(
        reducer = [reducer],
        geometry = geom,
        bands = [band],
        scale = scale,
        maxPixels = 1e10
    )
    serie = gee_ts_to_serie(ts, band).loc[:, band]
    return serie

# Cálculo de diferencia normalizada y filtrado
def normalized_difference(image, band1, band2, name = "ndwi"):
    index = image.normalizedDifference([band1, band2]).select(["nd"], [name])
    return index.set("system:time_start", image.get("system:time_start"))

def collection_normalized_difference(collection, band1, band2, name = "ndwi"):
    return collection.map(lambda image: normalized_difference(
        image, band1, band2, name = name))

def clip_collection(collection, geom):
    return collection.map(lambda image: image.clip(geom).set("system:time_start", 
                                                             image.get("system:time_start")))

def filter_collection(collection, gt = 0): 
    return collection.map(lambda image: image.gt(gt).selfMask().set("system:time_start", 
                                                                    image.get("system:time_start")))

# Cálculo de de área inundada
def compute_flooded_area(geometry, geom_area):
    cloud_percent = 5
    landsat5 = (ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
                  .filterDate("1984-03-16", "2012-05-05")
                  .filterBounds(geometry)
                  .filter(ee.Filter.lt("CLOUD_COVER", cloud_percent)));
    ndwi_l5 = collection_normalized_difference(landsat5, "SR_B2", "SR_B4", name = "ndwi")
    ndwi_l5_cliped = clip_collection(ndwi_l5, geometry)
    ndwi_l5_filtered = filter_collection(ndwi_l5_cliped, gt = 0)
    area_l5 = collection_timeserie(ndwi_l5_filtered, geometry, "ndwi", scale = 30, reducer = "count")
    total_area = collection_timeserie(ndwi_l5_cliped, geometry, "ndwi", scale = 30, reducer = "count")
    mask = ((total_area * 30 * 30) / geom_area) > 0.9
    area_l5 = area_l5.loc[mask]

    now = datetime.datetime.now().strftime("%Y-%m-%d")
    landsat8 = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
              .filterDate("2013-03-18", now)
              .filterBounds(geometry)
              .filter(ee.Filter.lt("CLOUD_COVER", cloud_percent)));
    ndwi_l8 = collection_normalized_difference(landsat8, "SR_B3", "SR_B5", name = "ndwi")
    ndwi_l8_cliped = clip_collection(ndwi_l8, geometry)
    ndwi_l8_filtered = filter_collection(ndwi_l8_cliped, gt = 0)
    area_l8 = collection_timeserie(ndwi_l8_filtered, geometry, "ndwi", scale = 30, reducer = "count")
    total_area = collection_timeserie(ndwi_l8_cliped, geometry, "ndwi", scale = 30, reducer = "count")
    mask = ((total_area * 30 * 30) / geom_area) > 0.9
    area_l8 = area_l8.loc[mask]

    area = pd.concat([area_l5, area_l8])
    return area.rename("area")*(30*30/10000)

# Generador de datos y obtención de serie de tiempo
def get_data(clave_presa):
    nombre_clave = clave[clave_presa]
    geometry = capa.filter(ee.Filter.eq(columna, nombre_clave)).first().geometry()

    serie = []
    try:
        area = compute_flooded_area(geometry, areas[nombre_clave])
    except:
        sleep(30)
        area = compute_flooded_area(geometry, areas[nombre_clave])
    sleep(1)
    serie.append(area)

    df_serie = pd.concat(serie)
    return df_serie

# Filtrado de datos
def fil_function (percent):
    datos = pd.Series(query["serie"])
    q1, q2, q3 = quantiles(datos, n = 4)
    ric = q3-q1
    if percent == 0.:
        limInf = q1-(3*ric)
        limSup = q3+(3*ric)
        datos_fil = datos.loc[(datos<limSup) & (datos>limInf)]
    elif percent == 5.:
        limInf = q1-(1.5*ric)
        limSup = q3+(1.5*ric)
        datos_fil = datos.loc[(datos<limSup) & (datos>limInf)]
    elif percent == 10.:
        limInf = q1-(1.25*ric)
        limSup = q3+(1.25*ric)
        datos_fil = datos.loc[(datos<limSup) & (datos>limInf)]
    elif percent == 15.:
        limInf = q1-(1*ric)
        limSup = q3+(1*ric)
        datos_fil = datos.loc[(datos<limSup) & (datos>limInf)]
    elif percent == 20.:
        limInf = q1-(0.75*ric)
        limSup = q3+(0.75*ric)
        datos_fil = datos.loc[(datos<limSup) & (datos>limInf)]
    elif percent == 25.:
        limInf = q1-(0.25*ric)
        limSup = q3+(0.25*ric)
        datos_fil = datos.loc[(datos<limSup) & (datos>limInf)]
    query['serie_fil'] = datos_fil
    return datos_fil

In [6]:
# @title Crear App

# Diccionario para guardar datos
query = {"serie": None, "serie_fil": None}

# Herramientas interactivas
clave_presa = pn.widgets.Select(
    name = "Presa a consultar",
    value = list(clave.keys())[0],
    options = list(clave.keys())
)

download_text = pn.pane.Markdown("")

guardar_como = pn.widgets.TextInput(name = "Guardar como", 
                                    placeholder = "Nombre de archivo de descarga...")

guardar_como = pn.widgets.TextInput(
    name = "Guardar como",
    placeholder = "Nombre del archivo a descargar..."
) 

filtro = pn.widgets.EditableFloatSlider(
    name = 'Prcentaje de filtrado', start = 0, 
    end = 25.0, step = 5.0, value = 5.0, disabled = True)

figure = go.Figure(
    data = go.Scatter(x=[], y=[])
)
fig_responsive = pn.pane.Plotly(figure, sizing_mode = 'stretch_width')

# Funciones de llamada
def on_button_clicked(b):
    button.name = "Procesando..."
    download_text.object = "Generando base de datos...."
    datos = get_data(clave_presa.value)
    query["serie"] = datos
    #actualizar figura
    figure = go.Figure(
        data = go.Scatter(
            x = datos.index,
            y = datos,
            mode = "lines"))
    figure.update_yaxes(title_text = 'Fecha')
    figure.update_yaxes(title_text = 'Área inundada [Ha]')
    figure.update_layout(
        height = 400,
        title = f'Presa "{clave_presa.value}"',
        title_x = 0.5,
        title_y = 0.95,
        showlegend = False)
    fig_responsive.object = figure
    # Actualizar etiquetas
    button.name = "Consultar datos"
    download_text.object = ""
    button_filtro.disabled = False
    filtro.disabled = False
    button_download.disabled = False

def up_filtro(b):
    download_text.objet = "Aplicando filtro..."
    datos_fil = fil_function (filtro.value)        
    #actualizar figura
    figure = go.Figure(
        data = go.Scatter(
            x = datos_fil.index,
            y = datos_fil,
            mode = "lines"))
    figure.update_yaxes(title_text = 'Fecha')
    figure.update_yaxes(title_text = 'Área inundada [Ha]')
    figure.update_layout(
        height = 400,
        title = f'Presa "{clave_presa.value}"',
        title_x = 0.5,
        title_y = 0.95,
        showlegend = False)
    fig_responsive.object = figure
    # Actualizar etiquetas
    download_text.object = ""
    button_download.disabled = False
    button_filtro.disabled = True

def up_rango(filtro):
    button_download.disabled = True
    if query["serie"] is None:
        button_filtro.disabled = True
    else:
        button_filtro.disabled = False
    

def download_serie():
    presicion = 5
    download = guardar_como.value
    datos = pd.Series(query["serie"])
    datos_fil = pd.Series(query["serie_fil"])
    if len(download) == 0:
        download = f"{clave_presa.value}"
    button_download.filename = f"{download}.csv"
    if datos_fil is not None:
        sio = StringIO()
        datos_fil.round(presicion).to_csv(sio)
        sio.seek(0)
        return sio
    else:
        sio = StringIO()
        datos.round(presicion).to_csv(sio)
        sio.seek(0)
        return sio

# Controles interactivos
button = pn.widgets.Button(name = "Consultar datos", button_type = "primary")
button_filtro = pn.widgets.Button(name = "Filtrar datos", button_type = "success", disabled=True)
button_download = pn.widgets.FileDownload(
    label = "Descargar serie",
    callback = pn.bind(download_serie),
    button_type = "warning",
    disabled = True)

button.on_click(on_button_clicked)
button_filtro.on_click(up_filtro)

# App principal
app = pn.Column(
    pn.Row(clave_presa, filtro),
    pn.Row(button, button_filtro),
    pn.Row(guardar_como, button_download),
    download_text,
    fig_responsive,
    pn.bind(up_rango, filtro),
    sizing_mode = 'stretch_width',   
)

app

Column(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [0] Row(design=<class 'panel.theme.materi...)
        [0] Select(design=<class 'panel.theme.materi..., name='Presa a consultar', options=['2 - Abelardo L. Rodrigue...], value='2 - Abelardo L...)
        [1] EditableFloatSlider(design=<class 'panel.theme.materi..., disabled=True, end=25.0, name='Prcentaje de filtrado', step=5.0, value=5.0, width=300)
    [1] Row(design=<class 'panel.theme.materi...)
        [0] Button(button_type='primary', design=<class 'panel.theme.materi..., name='Consultar datos')
        [1] Button(button_type='success', design=<class 'panel.theme.materi..., disabled=True, name='Filtrar datos')
    [2] Row(design=<class 'panel.theme.materi...)
        [0] TextInput(design=<class 'panel.theme.materi..., name='Guardar como', placeholder='Nombre del archivo a...)
        [1] FileDownload(button_type='warning', callback=<function bind.<locals>.wr..., design=<class 'panel.theme.materi..., disabled=True, label='Descargar serie')
    [3] Markdown(str, design=<class 'panel.theme.materi...)
    [4] Plotly(Figure, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [5] ParamFunction(function, _pane=Str, defer_load=False, design=<class 'panel.theme.materi...)